In [ ]:
! pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 36.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 3.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!pip install transformers[torch]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import pipeline


In [ ]:
#ATTEMPT 1: DO NOT USE
songs = pd.read_csv("chordProgs_semicolon.csv", sep=";")
songs = songs[:10000]

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

pipe = pipeline("text-generation", model=model,tokenizer=tokenizer)

def get_response(prompt: str) -> None:
    """
    Generate a response from the GPT2 model.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        None: Prints the model's response.
    """
    sequences = pipe(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=256,
        truncation=True,
    )
    print("Chatbot:", sequences[0]['generated_text'])

if False:
  #Tried this, but it fails becuase of the way these get split up into existing tokens anyway.
  new_tokens = ["I","II","III","IV","V","VI","VII",
                "i","ii","iii","iv","v","vi","vii",
                "I7","II7","III7","IV7","V7","VI7","VII7",
                "i7","ii7","iii7","iv7","v7","vi7","vii7",
                "Imaj7","IImaj7","IIImaj7","IVmaj7","Vmaj7","VImaj7","VIImaj7",
                "i07","ii07","iii07","iv07","v07","vi07","vii07",
                "bII7","bIII7","IV7","V7","bVI7","bVII7",
                "bII","bIII","IV","V","bVI","bVII",
                "bii7","biii7","#iv07","#v07","bvi7","bvii7",
                "bii","biii","#ivo","#vo","bvi","bvii"]

  new_tokens = ["I","i","V","v","0","o","7","maj","b","#"]


  # add to vocabulary
  tokenizer.add_tokens(list(new_tokens))

  # add embeddings for these tokens
  model.resize_token_embeddings(len(tokenizer))




train_data, test_data = train_test_split(songs, test_size = 0.2, random_state = 69)
#print(train_data,test_data)

#train_data_dict = {'Lyrics': [str(ele["Lyrics"]) if not pd.isnull(ele["Lyrics"]) else '' for _, ele in train_data.iterrows()]}
#train_data = Dataset.from_dict(train_data_dict)

#test_data_dict = {'Lyrics': [str(ele["Lyrics"]) if not pd.isnull(ele["Lyrics"]) else '' for _, ele in test_data.iterrows()]}
#test_data = Dataset.from_dict(test_data_dict)
train_data = Dataset.from_pandas(train_data)
test_data = Dataset.from_pandas(test_data)
print(test_data["Lyrics"])
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token


def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples["Lyrics"], max_length=100, padding="max_length", truncation=True)
    #tokenized_inputs = tokenizer(examples["Chords"], max_length=40, padding="max_length", truncation=True)
    chordProgs = tokenizer(examples["Chords"], max_length=40, padding="max_length", truncation=True)
    # Shift input ids to the right to create labels
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    #tokenized_inputs["labels"] = chordProgs["input_ids"].copy()
    return tokenized_inputs


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

[' "Hey, Miss Murder, can I? (Hey!) Hey, Miss Murder, can I? (Hey!) Make beauty stay if I take my life? (Hey!) Woah-oh Hey, Miss Murder, can I? (Hey!) Hey, Miss Murder, can I? (Hey!) Make beauty stay if I take my life? (Hey!) Woah-oh"', ' "Goodbye, Earl Those black-eyed peas? They tasted all right to me, Earl You\'re feeling weak? Why don\'t you lay down and sleep, Earl Ain\'t it dark? Wrapped up in that tarp, Earl"', ' "It seems the only blessing I have left to my name Is not knowing what we could have been What we should have been So"', ' "I wear a demeanor made of bright pretty things What she wears, what she wears, what she wears Birds singing on my shoulder in harmony it seems How they sing, how they sing, how they sing. I\'ve got halos made of summer, ribbons made of spring What she wears, what she wears, what she wears I\'ve got crowns of words a–woven, each one a song to sing Oh I sing, oh I sing, oh I sing"', ' "You keep jumpin\' \'round, in my face you know Like pushing me, i

In [ ]:
### ATTEMPT 2: NEW METHOD. DO NOT USE
songs =  pd.read_csv("train.csv", sep="\t")

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

pipe = pipeline("text-generation", model=model,tokenizer=tokenizer)

def get_response(prompt: str) -> None:
    """
    Generate a response from the GPT2 model.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        None: Prints the model's response.
    """
    sequences = pipe(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=256,
        truncation=True,
    )
    print("Chatbot:", sequences[0]['generated_text'])


train_data, test_data = train_test_split(songs, test_size = 0.2, random_state = 69)
print(test_data)

#train_data_dict = {'Lyrics': [str(ele["Lyrics"]) if not pd.isnull(ele["Lyrics"]) else '' for _, ele in train_data.iterrows()]}
#train_data = Dataset.from_dict(train_data_dict)

#test_data_dict = {'Lyrics': [str(ele["Lyrics"]) if not pd.isnull(ele["Lyrics"]) else '' for _, ele in test_data.iterrows()]}
#test_data = Dataset.from_dict(test_data_dict)
train_data = Dataset.from_pandas(train_data)
test_data = Dataset.from_pandas(test_data)
#print(test_data["text"])
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token


def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples["text"], max_length=500, padding="max_length", truncation=True)
    # Shift input ids to the right to create labels
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

                                                  text
47   ### Instruction: \nUsing roman numeral notatio...
68   ### Instruction: \nUsing roman numeral notatio...
181  ### Instruction: \nUsing roman numeral notatio...
57   ### Instruction: \nUsing roman numeral notatio...
89   ### Instruction: \nUsing roman numeral notatio...
58   ### Instruction: \nUsing roman numeral notatio...
126  ### Instruction: \nUsing roman numeral notatio...
32   ### Instruction: \nUsing roman numeral notatio...
95   ### Instruction: \nUsing roman numeral notatio...
162  ### Instruction: \nUsing roman numeral notatio...
31   ### Instruction: \nUsing roman numeral notatio...
86   ### Instruction: \nUsing roman numeral notatio...
153  ### Instruction: \nUsing roman numeral notatio...
178  ### Instruction: \nUsing roman numeral notatio...
23   ### Instruction: \nUsing roman numeral notatio...
8    ### Instruction: \nUsing roman numeral notatio...
13   ### Instruction: \nUsing roman numeral notatio...
15   ### I

In [ ]:
#DONT USE
lm_train_data = train_data.map(tokenize_function, batched=True, num_proc=4)
lm_test_data = test_data.map(tokenize_function, batched=True, num_proc=4)

train_valid_data = lm_train_data.train_test_split(test_size=0.2)
train_data = train_valid_data["train"]
valid_data = train_valid_data["test"]

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/163 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/41 [00:00<?, ? examples/s]

In [ ]:
#DON'T USE
training_args = TrainingArguments(
    "GPT2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=10
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=valid_data,
)

In [ ]:
#DON'T USE
trainer.train()

IndexError: index out of range in self

In [ ]:
#DON'T USE
text = tokenizer.batch_decode(label_ids)
#print(text)
for chordProg in text:
  print(chordProg.strip("<|endoftext|>"))

In [ ]:
##ATTEMPT 3, LOAD IN "chordProgs_semicolon.csv" AND USE THIS
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

#tokenizer.add_tokens(new_tokens)

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))
print(tokenizer.pad_token_id)

songs = pd.read_csv("chordProgs_semicolon.csv", sep=";") #Model 3.0
#songs = pd.read_csv("trainAttempt3.csv", sep=";",encoding="latin-1") #Model 3.1, another failed attempt
dataset = load_dataset("csv", data_files="chordProgs_semicolon.csv",sep=";")
#dataset = load_dataset("csv", data_files="trainAttempt3.csv",sep=";",encoding="latin-1")



train_data = dataset["train"].select([i for i in range(len(dataset["train"])) if i % 10 != 0])  # 9/10ths of songs for training
valid_data = dataset["train"].select([i for i in range(len(dataset["train"])) if i % 10 == 0]) # Only use every 10th song for testing.


def tokenize_function(instances):
    vLen = 512
    #progLen = 50
    inputs = tokenizer(instances['Lyrics'], return_tensors='pt', padding='max_length', max_length=vLen, truncation=True) #Model 3.0
    labels = tokenizer(instances['Chords'], return_tensors='pt', padding='max_length', max_length=vLen, truncation=True) #Model
    #inputs = tokenizer(instances['InputPrompt'], return_tensors='pt', padding='max_length', max_length=vLen, truncation=True)
    #labels = tokenizer(instances['Response'], return_tensors='pt', padding='max_length', max_length=vLen, truncation=True)
    inputs_trc = {}
    labels_trc = {}
    for k,v in inputs.items():
        v_trunc = v[:,:vLen]
        inputs_trc[k]=v_trunc
    for k,v in labels.items():
        v_trunc = v[:,:vLen]
        labels_trc[k]=v_trunc
    return {'input_ids': inputs_trc['input_ids'], 'labels': labels_trc['input_ids']}

train_data = train_data.map(tokenize_function, batched=True)
valid_data = valid_data.map(tokenize_function, batched=True)


print(train_data)
for row in valid_data:
  print(len(row["input_ids"]))
  print((row["input_ids"]))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


50257


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/183 [00:00<?, ? examples/s]

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Dataset({
    features: ['Genre', 'Title', 'Chords', 'Lyrics', 'input_ids', 'labels'],
    num_rows: 183
})
512
[366, 40, 1309, 340, 2121, 11, 616, 2612, 843, 355, 340, 3214, 11, 345, 8278, 284, 1624, 340, 632, 373, 3223, 290, 314, 373, 625, 14303, 345, 27946, 616, 11914, 290, 345, 7448, 502, 2011, 2832, 11, 484, 547, 1913, 887, 616, 14475, 547, 1290, 1165, 4939, 1675, 1302, 287, 534, 5101, 9170, 7463, 284, 534, 3625, 1649, 16299, 351, 345, 314, 714, 2652, 612, 13872, 616, 2951, 11, 1254, 345, 994, 8097, 921, 290, 502, 1978, 11, 2147, 318, 1365, 1, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 5025

In [ ]:
#TRAINER FOR ATTEMPT 3
training_args = TrainingArguments(
    output_dir='./task3out',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    evaluation_strategy="epoch",
    eval_steps=100,
    save_steps=500,
    logging_steps=100,
    #logging_dir='./task3Logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=valid_data,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.124475
2,0.206300,0.071479
3,0.060300,0.070289


TrainOutput(global_step=276, training_loss=0.11040511684141298, metrics={'train_runtime': 98.9519, 'train_samples_per_second': 5.548, 'train_steps_per_second': 2.789, 'total_flos': 143449325568000.0, 'train_loss': 0.11040511684141298, 'epoch': 3.0})

In [ ]:
import math
eval_results = trainer.evaluate()
print(eval_results)
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")
print("Perpexity score very misleading because of the padding. Real perplexity score would be in the hundreds.")

{'eval_loss': 0.07028878480195999, 'eval_runtime': 1.0955, 'eval_samples_per_second': 19.169, 'eval_steps_per_second': 2.738, 'epoch': 3.0}
Perplexity: 1.07


In [ ]:
pred = trainer.predict(valid_data)
predictions = pred.predictions
print(pred.predictions.shape)
label_ids = pred.label_ids
#print(predictions)
#print(label_ids)

(21, 512, 50258)


In [ ]:
import torch

for p in predictions:
  for wordProb in p:
    #print(wordProb)
    wordTensor = torch.from_numpy(wordProb)
    index = torch.argmax(wordTensor)
    word = tokenizer.decode(index)

    if word != "[PAD]":
      print(word, end="")
  print()

 I IV IV IV IV IV
 V IV IV IV
 I IV IV IV
 I I IV IV IV
 I IV b IV IV
 I b IV IV IV
 I b IV
 I b IV IV b
 I b IV IV
 I IV IV IV IV IV
 I IV IV IV
 I IV IV IV
 I IV IV IV
 I b IV IV
 V IV IV
 I I IV V
 I IV b IV
 I IV IV IV
 I IV IV IV IV
 I IV IV
 V IV IV IV


In [ ]:
#Inferencing on existing or made-up lyrics
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#lyrics = "I'm just a pig,\n working on the farm,\n when I came across a beehive,\n bigger than my arm\n"
lyrics = "You can take a ride in my big green tractor."
dataDict = {}
dataDict["Lyrics"] = lyrics
dataDict["Chords"] = "" #Unknown
input_ids = tokenize_function(dataDict)["input_ids"].to(device)
dataDict["input_ids"] = input_ids
attention_mask = torch.ones(input_ids.shape, device=device)
model = model.to(device)
model.eval()
result = model(torch.tensor(input_ids),attention_mask=attention_mask,)
print("Lyrics: "+lyrics)
print("Suggested Chords: ", end="")
for wordProb in result.logits[0]:
  #print(wordProb)
  index = torch.argmax(wordProb)
  word = tokenizer.decode(index)
  if word != "[PAD]":
    print(word, end="")
print()

Lyrics: You can take a ride in my big green tractor.
Suggested Chords:  b I


<ipython-input-21-3e7f2b61f222>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  result = model(torch.tensor(input_ids),attention_mask=attention_mask,)


In [ ]:
!pip install torcheval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 5.3 MB/s eta 0:00:00


In [ ]:
from torcheval.metrics.text import Perplexity
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

metric=Perplexity(device=device)
attention_mask = torch.ones(input_ids.shape, device=device)
model = model.to(device)
print(valid_data[0])

for song in valid_data:
  target = song["input_ids"]
  target = torch.tensor([[i] for i in target]).to(device)
  #print(target)
  result = model(target,attention_mask=attention_mask,).logits
  result = torch.tensor(result).to(device)
  #print(target)
  #print(result)
  metric.update(result, target)
  perpComponent = metric.compute()
  print(perpComponent)


{'Genre': 'Pop', 'Title': ' Set Fire to the Rain', 'Chords': ' i v bVII IV', 'Lyrics': ' "I let it fall, my heart And as it fell, you rose to claim it It was dark and I was over Until you kissed my lips and you saved me My hands, they were strong But my knees were far too weak To stand in your arms Without falling to your feet When laying with you I could stay there Close my eyes, feel you here forever You and me together, nothing is better"', 'input_ids': [366, 40, 1309, 340, 2121, 11, 616, 2612, 843, 355, 340, 3214, 11, 345, 8278, 284, 1624, 340, 632, 373, 3223, 290, 314, 373, 625, 14303, 345, 27946, 616, 11914, 290, 345, 7448, 502, 2011, 2832, 11, 484, 547, 1913, 887, 616, 14475, 547, 1290, 1165, 4939, 1675, 1302, 287, 534, 5101, 9170, 7463, 284, 534, 3625, 1649, 16299, 351, 345, 314, 714, 2652, 612, 13872, 616, 2951, 11, 1254, 345, 994, 8097, 921, 290, 502, 1978, 11, 2147, 318, 1365, 1, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257

<ipython-input-23-58bcb6d9a06f>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  result = torch.tensor(result).to(device)


tensor(3.2906e+09, device='cuda:0', dtype=torch.float64)
tensor(6.1571e+09, device='cuda:0', dtype=torch.float64)
tensor(5.7751e+09, device='cuda:0', dtype=torch.float64)
tensor(4.2686e+09, device='cuda:0', dtype=torch.float64)
tensor(5.0425e+09, device='cuda:0', dtype=torch.float64)
tensor(3.8229e+09, device='cuda:0', dtype=torch.float64)
tensor(4.2817e+09, device='cuda:0', dtype=torch.float64)
tensor(4.5863e+09, device='cuda:0', dtype=torch.float64)
tensor(4.6945e+09, device='cuda:0', dtype=torch.float64)
tensor(1.2249e+09, device='cuda:0', dtype=torch.float64)
tensor(1.2014e+09, device='cuda:0', dtype=torch.float64)
tensor(1.2834e+09, device='cuda:0', dtype=torch.float64)
tensor(1.2610e+09, device='cuda:0', dtype=torch.float64)
tensor(1.3726e+09, device='cuda:0', dtype=torch.float64)
tensor(1.5848e+09, device='cuda:0', dtype=torch.float64)
tensor(1.7081e+09, device='cuda:0', dtype=torch.float64)
tensor(1.9823e+09, device='cuda:0', dtype=torch.float64)
tensor(2.1020e+09, device='cuda

In [ ]:
#Inferencing for Precision, Recall, and Accuracy calculation
#Sample provided is sections from 9 songs, at least 2 from each of the 4 genres.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lyricList = ["And I can take you for a ride on my big green tractor We can go slow or make it go faster Down through the woods and out to the pasture Long as I'm with you, it really don't matter",
             "I wanna rock and roll all night and party every day",
             "With a taste of a poison paradise, I'm addicted to you. Don't you know that you're toxic?",
             "I'm Slim Shady, yes, I'm the real Shady All you other Slim Shadys are just imitating",
             "Country Roads, take me home, to the place I belong",
             "Lie to my face. Tell me it ain't no thing, that's what I wanna hear",
             "They say that time's supposed to heal you, but I ain't done much healing.",
             "Sun is down, freezin' cold. That's how we already know, winter's here.",
             "You can't kill the metal. The metal will live on.",
             "Two trailer park girls go round the outside,round the outside, round the outside,"]
chords = ["I V ii IV V", "I I IV V", "i bIII II bII", "i iv bIII V", "I V vi IV","i bVII V","vi I V IV","Imaj7 VII Imaj7 VII","i i i bII","i V bVI V"]
for i, lyrics in enumerate(lyricList):
  dataDict = {}
  dataDict["Lyrics"] = lyrics
  dataDict["Chords"] = "" #Unknown
  input_ids = tokenize_function(dataDict)["input_ids"].to(device)
  dataDict["input_ids"] = input_ids
  attention_mask = torch.ones(input_ids.shape, device=device)
  model = model.to(device)
  model.eval()
  result = model(torch.tensor(input_ids),attention_mask=attention_mask,)
  print("Lyrics: "+lyrics)
  print("Suggested Chords: ", end="")
  for wordProb in result.logits[0]:
    #print(wordProb)
    index = torch.argmax(wordProb)
    word = tokenizer.decode(index)
    if word != "[PAD]":
      print(word, end="")
  print()
  print("Real Chords: ", chords[i])

<ipython-input-30-48b91bcd7543>:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  result = model(torch.tensor(input_ids),attention_mask=attention_mask,)


Lyrics: And I can take you for a ride on my big green tractor We can go slow or make it go faster Down through the woods and out to the pasture Long as I'm with you, it really don't matter
Suggested Chords:  I IV IV
Real Chords:  I V ii IV V
Lyrics: I wanna rock and roll all night and party every day
Suggested Chords:  V IV I
Real Chords:  I I IV V
Lyrics: With a taste of a poison paradise, I'm addicted to you. Don't you know that you're toxic?
Suggested Chords:  V b
Real Chords:  i bIII II bII
Lyrics: I'm Slim Shady, yes, I'm the real Shady All you other Slim Shadys are just imitating
Suggested Chords:  V V
Real Chords:  i iv bIII V
Lyrics: Country Roads, take me home, to the place I belong
Suggested Chords:  V IV
Real Chords:  I V vi IV
Lyrics: Lie to my face. Tell me it ain't no thing, that's what I wanna hear
Suggested Chords:  V IV
Real Chords:  i bVII V
Lyrics: They say that time's supposed to heal you, but I ain't done much healing.
Suggested Chords:  b I I
Real Chords:  vi I V 

In [ ]:
#PRECISION, RECALL, F1Score Calculations done by hand here
#Inferenced song #:     1    2    3    4    5    6    7    8    9   10
#Recall:               0.4   0.25 0    0.25 0.5  0.33 0.25 0    0   0.5
#Precision:             1    0.5  0    1    1    0.5  0.67 0    0   0.5
#F1Score(2PR/(P+R)):  0.57  0.33 0    0.4  0.33 0.4  0.36 0    0   0.5

#AVG RECALL = 0.248
#AVG PRECISION = 0.517
#AVG F1Score = 0.289
#Precision is only as high as it is because the model prefers such short progressions.